# LSTM

In [29]:
import pandas as pd
import os
import plotly.express as px
import json
import numpy as np
from utils import get_itslive, get_processed_data, get_future_dates
from tqdm import tqdm
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler

In [30]:
def setConfig():
    """
    Configura el dispositivo para PyTorch y spaCy según la disponibilidad de hardware.
    Devuelve el dispositivo configurado.
    """
    if torch.backends.mps.is_available():
        # Usar MPS (Metal Performance Shaders) en macOS
        device = torch.device("mps")
        print("Usando MPS:", device)
    elif torch.cuda.is_available():
        # Usar CUDA (GPU NVIDIA) si está disponible
        device = torch.device("cuda")
        os.environ["CUDA_VISIBLE_DEVICES"] = "0"
        os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"
        os.environ["TOKENIZERS_PARALLELISM"] = "false"
        os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
        print("Usando CUDA:", device)
    else:
        # Usar CPU como fallback
        device = torch.device("cpu")
        print("Usando CPU:", device)
    
    # Configurar TQDM en pandas
    tqdm.pandas()

    try:
        torch.ones(1, device=device)
        print("Test correcto:", device)
    except Exception as e:
        print("Error al configurar el dispositivo:", e)
        raise e
    
    return device

device = setConfig()

Usando MPS: mps
Test correcto: mps


In [31]:
with open("glaciares.json") as f:
    glaciares = json.load(f)

coords = glaciares["Groenlandia - Sermeq Kujalleq [Jakobshavn Isbræ]"]

df = get_itslive([coords])
glacier = get_processed_data(df)

original xy [-49.55383, 69.13788] 4326 maps to datacube (-181358.1596550405, -2277021.305723809) EPSG:3413


/Users/roicort/GitHub/F-VICE/its_live/datacube_tools.py:283: FutureWarning:

In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.



In [32]:
glacier

,v,v_error,vx,vx_error,vy,vy_error,date_dt,satellite_img1,mission_img1,x,y,lat,lon,year,month,dayofyear
mid_date,,,,,,,,,,,,,,,,
1985-04-06 14:30:57.954894976+00:00,5340,196.0,-3604.0,173.500000,3940.0,212.500000,15 days 23:59:55.220947265,5,L,-181387.5,-2277052.5,69.13788,-49.55383,1985,4,96
1985-04-11 02:27:50.807382016+00:00,5044,178.0,-3362.0,138.500000,3760.0,204.000000,24 days 23:53:40.971679687,5,L,-181387.5,-2277052.5,69.13788,-49.55383,1985,4,101
1985-04-14 14:30:54.476366976+00:00,4675,144.0,-3017.0,123.900002,3571.0,157.199997,31 days 23:59:48.299560547,5,L,-181387.5,-2277052.5,69.13788,-49.55383,1985,4,104
1985-04-18 02:33:57.912401024+00:00,4425,98.0,-2737.0,75.900002,3477.0,110.199997,39 days 00:05:54.968261719,5,L,-181387.5,-2277052.5,69.13788,-49.55383,1985,4,108
1985-04-22 14:30:52.080955072+00:00,4754,305.0,-2967.0,249.600006,3714.0,336.000000,15 days 23:59:53.078613281,5,L,-181387.5,-2277052.5,69.13788,-49.55383,1985,4,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-02 15:19:49.240922112+00:00,13620,32.0,-9607.0,24.000000,9655.0,38.000000,20 days 00:02:20.075683593,2B,S,-181387.5,-2277052.5,69.13788,-49.55383,2024,10,276
2024-10-03 03:30:25.240924928+00:00,13682,32.0,-9571.0,25.400000,9778.0,37.500000,15 days 00:02:32.023315429,2B,S,-181387.5,-2277052.5,69.13788,-49.55383,2024,10,277
2024-10-03 14:54:21.893806080+00:00,13648,40.0,-9514.0,36.099998,9785.0,43.000000,16 days 00:00:06.921386718,8,L,-181387.5,-2277052.5,69.13788,-49.55383,2024,10,277


In [33]:
X = glacier[['year', 'month', 'dayofyear']].copy()
y = glacier['v']

X['v_lag1'] = y.shift(1)
X['v_lag7'] = y.shift(7)
X['v_rollmean7'] = y.shift(1).rolling(window=7).mean()
X['v_diff1'] = y.diff(1)
X['v_diff7'] = y.diff(7)

valid_idx = X.dropna().index
X = X.loc[valid_idx]
y = y.loc[valid_idx]

split_idx = int(len(X) * 0.70)

X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

In [34]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1)).flatten()

def create_sequences(X, y, seq_len):
    Xs, ys = [], []
    for i in range(len(X) - seq_len):
        Xs.append(X[i:i+seq_len])
        ys.append(y[i+seq_len])
    return np.array(Xs), np.array(ys)

seq_len = 3  # Puedes ajustar este valor

X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, seq_len)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, seq_len)

# Convertir a tensores
X_train_ = torch.tensor(X_train_seq, dtype=torch.float)
y_train_ = torch.tensor(y_train_seq, dtype=torch.float)
X_test_ = torch.tensor(X_test_seq, dtype=torch.float)
y_test_ = torch.tensor(y_test_seq, dtype=torch.float)

# Datasets y dataloaders
train_data = TensorDataset(X_train_, y_train_)
test_data = TensorDataset(X_test_, y_test_)

batch_size = 128
train_dataloader = DataLoader(train_data, shuffle=False, batch_size=batch_size)
test_dataloader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

In [39]:
from neural import DenseLSTM

input_dim = 8
hidden_dim = 64 
epochs = 30

model = DenseLSTM(input_dim, hidden_dim, lstm_layers=2, bidirectional=True, dense=True)
model.to(device)

optimizer = torch.optim.Adam(model.parameters())
criterion = nn.MSELoss()

model.fit(
        train_dataloader,
        test_dataloader,
        optimizer,
        criterion,
        device,
        epochs=epochs,
    )

  0%|          | 0/30 [00:00<?, ?it/s]

100%|██████████| 30/30 [00:07<00:00,  4.26it/s, train_loss=0.0566, test_loss=0.0198, time=0.18] 


In [40]:
model.eval()

y_pred = model.predict(
    test_dataloader,
    device,
)

In [41]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

y_pred = scaler_y.inverse_transform(y_pred.reshape(-1, 1)).flatten()
y_test_seq_inv = scaler_y.inverse_transform(y_test_seq.reshape(-1, 1)).flatten()

mse = mean_squared_error(y_test_seq_inv, y_pred)
mae = mean_absolute_error(y_test_seq_inv, y_pred)

r2 = r2_score(y_test_seq_inv, y_pred)
n = X_test.shape[0]  # número de muestras
p = X_test.shape[1]  # número de predictores
r2_adj = 1 - (1 - r2) * (n - 1) / (n - p - 1)

results = {
    "MAE": [mae],
    "MSE": [mse],
    "RMSE": [np.sqrt(mse)],
    "R2": [r2],
    "R2_adj": [r2_adj]
}
print(results)
results_df = pd.DataFrame(results)
results_df.T

{'MAE': [214.37082093965023], 'MSE': [133151.21999739748], 'RMSE': [np.float64(364.8989175064753)], 'R2': [0.9650416268661339], 'R2_adj': [0.9647836314924523]}


,0
MAE,214.370821
MSE,133151.219997
RMSE,364.898918
R2,0.965042
R2_adj,0.964784


In [42]:
future_dates = get_future_dates(X_test.index[-1], until='2030-12-31')
future_features = future_dates[['year', 'month', 'dayofyear']].copy()

# Inicializa las listas con los últimos valores reales
last_known = glacier.loc[X_test.index[-1]]
history = list(y.values[-30:])  # Últimos 30 valores reales

for i in range(len(future_features)):
    # Calcular lags y medias móviles usando el historial
    v_lag1 = history[-1]
    v_lag7 = history[-7] if len(history) >= 7 else np.nan
    v_lag30 = history[-30] if len(history) >= 30 else np.nan
    v_rollmean7 = np.mean(history[-7:]) if len(history) >= 7 else np.nan
    v_rollmean30 = np.mean(history[-30:]) if len(history) >= 30 else np.nan
    v_diff1 = history[-1] - history[-2] if len(history) >= 2 else np.nan
    v_diff7 = history[-1] - history[-7] if len(history) >= 7 else np.nan

    # Añadir features al DataFrame
    future_features.loc[future_features.index[i], 'v_lag1'] = v_lag1
    future_features.loc[future_features.index[i], 'v_lag7'] = v_lag7
    future_features.loc[future_features.index[i], 'v_rollmean7'] = v_rollmean7
    future_features.loc[future_features.index[i], 'v_diff1'] = v_diff1
    future_features.loc[future_features.index[i], 'v_diff7'] = v_diff7

    # Preprocesa y predice el siguiente valor
    X_future_scaled = scaler_X.transform(future_features.iloc[[i]])
    X_future_tensor = torch.tensor(X_future_scaled, dtype=torch.float).unsqueeze(0).to(device)
    with torch.no_grad():
        pred_scaled = model(X_future_tensor).cpu().numpy().flatten()[0]
    pred = scaler_y.inverse_transform([[pred_scaled]])[0, 0]

    # Añade la predicción al historial para los siguientes pasos
    history.append(pred)

# Predecir 

future_features['v'] = history[-len(future_features):]
future_predictions = future_features['v'].values

In [43]:
# Plot predictions

plot_df = pd.DataFrame({
    'v': pd.concat([y_train, y_test]),
    'split': ['train'] * len(X_train) + ['test'] * len(X_test)
})


# Graficar puntos reales y predicciones
fig = px.line(
    plot_df,
    x=plot_df.index,
    y='v',
    color='split',
    title='Train/Test Split & Predictions: v over Time',
)

# Agregar las predicciones como línea
fig.add_scatter(
    x=y_test.index,
    y=y_pred,
    mode='lines',
    name='Predicción (test)',
    line=dict(color="#4e59f6", width=2),
)

fig.show()

# Agregar las predicciones futuras
fig.add_scatter(
    x=future_dates.index,
    y=future_predictions,
    mode='lines',
    name='Predicción (futuro)',
    line=dict(color="#e006bf", width=2),
)

fig.show()

In [ ]:
fig.write_image(
    os.path.join("latex/svg-inkscape", f"{'Jakobshavn'}_LSTM_forecast.svg"),
    width=1000,
    height=500,
)